In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
class AssismentData():
    def __init__(self):
        self.data = pd.read_csv("/content/drive/My Drive/DKT/skill_builder_data.csv")
        self.data = self.data.dropna()
        self.data["user_id"], _ = pd.factorize(self.data["user_id"])
        self.data["skill_id"], _ = pd.factorize(self.data["skill_id"])
        self.data["skills_correctness"] = self.data.apply(
            lambda x: x.skill_id * 2 if x.correct == 0.0 else x.skill_id * 2 + 1, axis=1)
        self.data = self.data.groupby("user_id").filter(lambda q: len(q) > 1).copy()
        self.seq = self.data.groupby('user_id').apply(
            lambda r: (
                r["skills_correctness"].values[:-1],
                r["skill_id"].values[1:],
                r['correct'].values[1:]
            )
        )

    def datasetReturn(self, shuffle=None, batch_size=32, val_data=None):

        dataset = tf.data.Dataset.from_generator(lambda: self.seq, output_types=(tf.int32, tf.int32, tf.int32))

        if shuffle:
            dataset = dataset.shuffle(buffer_size=shuffle)

        MASK_VALUE = -1
        dataset = dataset.padded_batch(
            batch_size=50,
            padding_values=(MASK_VALUE,MASK_VALUE, MASK_VALUE),
            padded_shapes=([None], [None], [None]),
            drop_remainder=True
        )
        i = 0
        for l in dataset.as_numpy_iterator():
            i += 1
 
        dataset = dataset.shuffle(buffer_size=50)
        test_size = int(np.ceil(i * 0.2))
        train_size = i - test_size
      
        train_data = dataset.take(train_size)
        dataset = dataset.skip(train_size)

        return train_data, dataset

In [2]:
ass = AssismentData()
train_data,test_data = ass.datasetReturn()
val_log = 'log/val'
train_loss_log = 'log/train'
summary_writer = tf.summary.create_file_writer(val_log)

In [3]:
print(ass.seq.head(5))

user_id
0    ([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,...
1    ([2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 5, 5, 5, 6, 8,...
2    ([6, 14, 7, 9, 6, 8, 7, 9, 7, 9, 7, 17, 7, 17,...
4    ([37, 34, 35, 39, 39, 49, 45, 43, 45, 45, 37, ...
5    ([37, 41, 35, 39, 37, 37, 37, 35, 37, 35, 43, ...
dtype: object


In [3]:
class DKT(tf.keras.models.Model):
    def __init__(self, total_skills_correctness, embedding_size):
        super(DKT, self).__init__(name="DKTModel")

        self.mask = tf.keras.layers.Masking(mask_value=-1)

        # 两个嵌入层
       
        self.skill_embedding = tf.keras.layers.Embedding(total_skills_correctness, embedding_size)
        # RNN
        self.rnn = tf.keras.Sequential ( [tf.keras.layers.LSTM(units=64, return_sequences=True, dropout=0.3),
                            tf.keras.layers.LSTM(units=64, return_sequences=True, dropout=0.3)])

        # dense
        self.dense = tf.keras.layers.Dense(total_skills_correctness/2, activation='sigmoid')

        self.distribute = tf.keras.layers.TimeDistributed(self.dense)

        self.softmax = tf.keras.layers.Softmax()

    def call(self, skillid):
     
        skillid = tf.expand_dims(skillid, axis=-1)

        skillid = self.mask(skillid)


        skill_vector = self.skill_embedding(skillid)

        x = skill_vector

        x = tf.squeeze(x, axis=-2)
        x = self.rnn(x)
        y = self.distribute(x)

        return y

In [4]:
dkt = DKT(int(ass.data["skills_correctness"].max() + 1), 32)
skill_num = int((ass.data["skills_correctness"].max() + 1)/2)
print(skill_num)
AUC = tf.keras.metrics.AUC()
VAUC = tf.keras.metrics.AUC()
SCC = tf.keras.metrics.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
def test_one_step(sequence_id,skillid, label):
    loss = dkt(skillid)

    label = tf.expand_dims(label, axis=-1)

    mask = 1. - tf.cast(tf.equal(label, -1),tf.float32)
    
    mask = tf.squeeze(mask)

    squenceid = tf.boolean_mask(sequence_id, mask=mask)
    squenceid = tf.one_hot(squenceid,depth=skill_num,axis=-1)
    label = tf.boolean_mask(label, mask=mask)
    loss = tf.boolean_mask(loss, mask=mask)

    loss = tf.expand_dims(tf.reduce_sum(tf.multiply(squenceid , loss),axis=-1),axis=-1)

    loss = tf.expand_dims(loss,axis=-1)

    label = tf.squeeze(tf.one_hot(label,depth=2),axis=1)

    loss = tf.concat([1-loss,loss],axis=1)
    
    VAUC.update_state(label , tf.squeeze(loss,axis=-1))


def train_one_step(sequence_id,skillid, label):
    with tf.GradientTape() as tape:
        loss = dkt(skillid)
        
        label = tf.expand_dims(label, axis=-1)
  
        mask = 1. - tf.cast(tf.equal(label, -1),tf.float32)
        mask = tf.squeeze(mask)

        sequenceid = tf.boolean_mask(sequence_id, mask=mask)
        sequenceid = tf.one_hot(sequenceid,depth=skill_num,axis=-1)
        label = tf.boolean_mask(label, mask=mask)
        loss = tf.boolean_mask(loss, mask=mask)

     
        loss = tf.expand_dims(tf.reduce_sum(tf.multiply(sequenceid,loss),axis=-1),axis=-1)
    
        loss_real = tf.reduce_sum(tf.keras.losses.binary_crossentropy(label , loss))
        
        SCC.update_state(label, loss)
  
        loss = tf.expand_dims(loss,axis=-1)
        
        loss = tf.concat([1-loss,loss],axis=1)
        
        label = tf.squeeze(tf.one_hot(label,depth=2),axis=1)
  
        AUC.update_state(label , tf.squeeze(loss,axis=-1))
        
        gradients = tape.gradient(loss_real, dkt.trainable_variables)
        # 反向传播，自动微分计算
        optimizer.apply_gradients(zip(gradients, dkt.trainable_variables))

123


In [5]:
for epoch in range(8):
  train_data = train_data.shuffle(50)
  AUC.reset_states()
  VAUC.reset_states()
  SCC.reset_states()
  for s,p,l in train_data.as_numpy_iterator():
    train_one_step(p,s,l)

  for s,p,l in test_data.as_numpy_iterator():
    test_one_step(p,s,l)

  with summary_writer.as_default():
    tf.summary.scalar('train_auc',AUC.result(),step=epoch)
    tf.summary.scalar('val_auc',VAUC.result(),step=epoch)
    
  print(SCC.result(),AUC.result(),VAUC.result())

tf.Tensor(0.64549065, shape=(), dtype=float32) tf.Tensor(0.6776958, shape=(), dtype=float32) tf.Tensor(0.7657523, shape=(), dtype=float32)
tf.Tensor(0.56331336, shape=(), dtype=float32) tf.Tensor(0.7785327, shape=(), dtype=float32) tf.Tensor(0.8198595, shape=(), dtype=float32)
tf.Tensor(0.49902382, shape=(), dtype=float32) tf.Tensor(0.83314186, shape=(), dtype=float32) tf.Tensor(0.8292503, shape=(), dtype=float32)
tf.Tensor(0.48466077, shape=(), dtype=float32) tf.Tensor(0.84098864, shape=(), dtype=float32) tf.Tensor(0.8500039, shape=(), dtype=float32)
tf.Tensor(0.47161525, shape=(), dtype=float32) tf.Tensor(0.8497119, shape=(), dtype=float32) tf.Tensor(0.8409797, shape=(), dtype=float32)
tf.Tensor(0.46274725, shape=(), dtype=float32) tf.Tensor(0.8558189, shape=(), dtype=float32) tf.Tensor(0.8516734, shape=(), dtype=float32)
tf.Tensor(0.46378323, shape=(), dtype=float32) tf.Tensor(0.85513514, shape=(), dtype=float32) tf.Tensor(0.8352248, shape=(), dtype=float32)
tf.Tensor(0.46658456, sh